In [ ]:
import numpy as np
import timeit

import matplotlib.pyplot as plt

import torch
import torch.optim as optim

import fitsio 

import sys
sys.path.insert(0, './../')
import sdss_psf
import simulated_datasets_lib
import starnet_vae_lib
import sdss_dataset_lib
import plotting_utils
import image_statistics_lib
import utils

import inv_kl_objective_lib as inv_kl_lib

import image_utils

import time

import json

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('torch version: ', torch.__version__)

from copy import deepcopy

In [ ]:
np.random.seed(22)
_ = torch.manual_seed(22)

# Load data

In [ ]:
sdss_hubble_data = sdss_dataset_lib.SDSSHubbleData(bands = [2, 3], 
                                                  background_bias = torch.Tensor([168., 222.]))

In [ ]:
f_min = 1000.

In [ ]:
# the oberved data 
sdss_images_full = sdss_hubble_data.sdss_image.unsqueeze(0)

# get true parameters
backgrounds_full = sdss_hubble_data.sdss_background.unsqueeze(0)

which_bright = sdss_hubble_data.fluxes[:, 0] > f_min
true_full_locs = sdss_hubble_data.locs[which_bright].unsqueeze(0)
true_full_fluxes = sdss_hubble_data.fluxes[which_bright].unsqueeze(0)

In [ ]:
backgrounds_full.mean(-1).mean(-1)

In [ ]:
# get PSFs 
psf_dir = '../data/'
psf_r = fitsio.FITS(psf_dir + 'sdss-002583-2-0136-psf-r.fits')[0].read()
psf_i = fitsio.FITS(psf_dir + 'sdss-002583-2-0136-psf-i.fits')[0].read()

psf_og = np.array([psf_r, psf_i])
sky_intensity = torch.Tensor([854., 1345.])

In [ ]:
simulator = simulated_datasets_lib.StarSimulator(psf_og, sdss_images_full.shape[-1], 
                                                 sky_intensity, 
                                                 transpose_psf = False)

# Load VAE

In [ ]:
star_encoder = starnet_vae_lib.StarEncoder(full_slen = sdss_images_full.shape[-1], 
                                            stamp_slen = 7,
                                            step = 2,
                                            edge_padding = 2, 
                                            n_bands = 2,
                                            max_detections = 2)

In [ ]:
star_encoder.load_state_dict(torch.load('../fits/results_2020-01-21/starnet_ri',
                               map_location=lambda storage, loc: storage))
star_encoder.eval(); 

# OK first, let us check results on data simulated using hubble parameters

In [ ]:
# simulate data using hubble parameters
# two options for here: can either draw from bright stars only or all the stars

# sim_images_full = simulator.draw_image_from_params(locs = sdss_hubble_data.locs.unsqueeze(0), 
#                         fluxes = sdss_hubble_data.fluxes.unsqueeze(0), 
#                         n_stars = torch.Tensor([len(sdss_hubble_data.locs)]).type(torch.LongTensor), 
#                         add_noise = True) 

sim_images_full = simulator.draw_image_from_params(locs = true_full_locs,
                        fluxes = true_full_fluxes,
                        n_stars = torch.Tensor([true_full_locs.shape[1]]).type(torch.LongTensor), 
                        add_noise = True) 

In [ ]:
# check residuals

In [ ]:
band = 0

In [ ]:
f, axarr = plt.subplots(1, 3, figsize=(16, 6))

im0 = axarr[0].matshow(sdss_images_full[0, band]); 
f.colorbar(im0, ax = axarr[0])
axarr[0].set_title('true sdss image')

im1 = axarr[1].matshow(sim_images_full[0, band]); 
f.colorbar(im1, ax = axarr[1])
axarr[1].set_title('simulated sdss image')


residual = torch.log10(sim_images_full[0, band]) - torch.log10(sdss_images_full[0, band])
vmax = residual[10:90, 10:90].abs().max()
im2 = axarr[2].matshow(residual[10:90, 10:90], vmax = vmax, vmin = -vmax, cmap=plt.get_cmap('bwr')); 
f.colorbar(im2, ax = axarr[2])
axarr[2].set_title('residual')

## Check loss

In [ ]:
# check loss 
loss, counter_loss, locs_loss, fluxes_loss, perm_indx = \
    inv_kl_lib.get_encoder_loss(star_encoder, sim_images_full, backgrounds_full, 
                                true_full_locs, true_full_fluxes)[0:5]

In [ ]:
print('loss: {:06f}'.format(loss))

In [ ]:
print(counter_loss.mean())
print(locs_loss.mean())
print(fluxes_loss.mean())

## Check reconstructions

In [ ]:
# get parameters on the simulated image 
map_locs_sim_image, map_fluxes_sim_image, map_n_stars_sim_image = \
        star_encoder.sample_star_encoder(sim_images_full, backgrounds_full, 
                                        return_map_n_stars = True, 
                                        return_map_star_params = True)[0:3]

In [ ]:
# OK this is silly but lets try this: 
from torch import nn
class EstimateFluxes(nn.Module): 
    def __init__(self, observed_image, slen, locs, 
                         psf, sky_intensity, init_fluxes): 
        
        super(EstimateFluxes, self).__init__()
        
        assert len(locs.shape) == 2
        assert len(init_fluxes.shape) == 2
        assert locs.shape[0] == init_fluxes.shape[0]
        assert len(observed_image.shape) == 3
        
        self.n_bands = psf.shape[0]
        assert observed_image.shape[0] == self.n_bands        
        assert init_fluxes.shape[1] == self.n_bands
        assert len(sky_intensity) == self.n_bands
        
        self.observed_image = observed_image 
        
        self.sky_intensity = sky_intensity
        
        self.star_basis = simulated_datasets_lib.plot_one_star(slen, locs, psf)
            
        self.log_flux = nn.Parameter(torch.log(init_fluxes).clone())
        
    def forward(self): 
        return (torch.exp(self.log_flux[:, :, None, None]) * flux_estimator.star_basis).sum(0)
    
    def get_loss(self): 
        recon_mean = self.forward() + self.sky_intensity[:, None, None]
        return ((self.observed_image - recon_mean)**2 / recon_mean).mean()
    
    def optimize(self): 
        optimizer = optim.LBFGS(self.parameters())
        
        def closure():
            optimizer.zero_grad()
            loss = self.get_loss()
            loss.backward()
            return loss
        
        optimizer.step(closure)

In [ ]:
flux_estimator.sky_intensity

In [ ]:
if True: 
    flux_estimator = EstimateFluxes(sim_images_full.squeeze(0), 
                            star_encoder.full_slen, 
                            map_locs_sim_image.squeeze(0), 
                            simulator.psf, 
                            sky_intensity, 
                            map_fluxes_sim_image.squeeze(0))
    
    print(flux_estimator.get_loss())
    flux_estimator.optimize()
    print(flux_estimator.get_loss())
    map_fluxes_sim_image = torch.exp(flux_estimator.log_flux.data.unsqueeze(0))

In [ ]:
plt.hist(torch.log10(map_fluxes_sim_image[0, :, 0]))

In [ ]:
# get reconstructed mean
vae_recon_mean = simulator.draw_image_from_params(locs = map_locs_sim_image, 
                                                fluxes = map_fluxes_sim_image,
                                                 n_stars = map_n_stars_sim_image, 
                                                 add_noise = False).detach()

In [ ]:
band

In [ ]:
fig, axarr = plt.subplots(1, 3, figsize=(15, 4))
im0 = axarr[0].matshow(sim_images_full[0, band][5:95, 5:95])
fig.colorbar(im0, ax = axarr[0])

im1 = axarr[1].matshow(vae_recon_mean[0, band][5:95, 5:95])
fig.colorbar(im1, ax = axarr[1])

residual = torch.log10(vae_recon_mean[0, band]) - torch.log10(sim_images_full[0, band])
_residual = (residual * 2.5)[5:95, 5:95]
# (torch.log(vae_recon_mean.squeeze()) - torch.log(images_full.squeeze()))[10:90, 10:90]
vmax = _residual.abs().max()
im2 = axarr[2].matshow(_residual, vmax = vmax, vmin = -vmax, cmap=plt.get_cmap('bwr'))
fig.colorbar(im2, ax = axarr[2])

In [ ]:
((vae_recon_mean[0, band, 5:95, 5:95] - sim_images_full[0, band, 5:95, 5:95])**2).sum()

In [ ]:
def get_which_tile(x0, x1, tile_coords, edge_padding, stamp_slen): 
    coords = tile_coords + edge_padding
    
    view_slen = stamp_slen - 2 * edge_padding
    
    indx = torch.where((x0 > coords[:, 0]) & \
                       (x0 < coords[:, 0] + view_slen) & \
                       (x1 > coords[:, 1]) & \
                       (x1 < coords[:, 1] + view_slen))
        
    return tile_coords[indx], indx

In [ ]:
get_which_tile(70, 58, star_encoder.tile_coords, star_encoder.edge_padding, star_encoder.stamp_slen)

In [ ]:
f, axarr = plt.subplots(1, 3, figsize=(16, 4))

indx = 1611 # int(np.random.choice(star_encoder.tile_coords.shape[0], 1))

plotting_utils.plot_subimage(axarr[0], sim_images_full[0, band],
                            map_locs_sim_image.squeeze(), 
                            true_full_locs.squeeze(), 
                            int(star_encoder.tile_coords[indx, 0]), 
                            int(star_encoder.tile_coords[indx, 1]), 
                            subimage_slen = star_encoder.stamp_slen, 
                            add_colorbar = True, 
                            global_fig = f)

plotting_utils.plot_subimage(axarr[1], vae_recon_mean[0, band],
                            map_locs_sim_image.squeeze(), 
                            None, 
                            int(star_encoder.tile_coords[indx, 0]), 
                            int(star_encoder.tile_coords[indx, 1]), 
                            subimage_slen = star_encoder.stamp_slen, 
                            add_colorbar = True, 
                            global_fig = f)

foo = torch.log10(vae_recon_mean[0, band]) - torch.log10(sim_images_full[0, band])
plotting_utils.plot_subimage(axarr[2], foo * 2.5, 
                            map_locs_sim_image.squeeze(), 
                            None, 
                            int(star_encoder.tile_coords[indx, 0]), 
                            int(star_encoder.tile_coords[indx, 1]), 
                            subimage_slen = star_encoder.stamp_slen, 
                            add_colorbar = True, 
                            global_fig = f, 
                            diverging_cmap = True)

axarr[0].axvline(x=2, color = 'r')
axarr[0].axvline(x=4, color = 'r')
axarr[0].axhline(y=2, color = 'r')
axarr[0].axhline(y=4, color = 'r')

axarr[1].axvline(x=2, color = 'r')
axarr[1].axvline(x=4, color = 'r')
axarr[1].axhline(y=2, color = 'r')
axarr[1].axhline(y=4, color = 'r')

## Check summary statistics

In [ ]:
n_elect_per_nmgy = sdss_hubble_data.nelec_per_nmgy.mean()

In [ ]:
completeness, tpr, completeness_bool, tpr_bool = \
    image_statistics_lib.get_summary_stats(map_locs_sim_image.squeeze(), 
                                           true_full_locs.squeeze(), 
                                           star_encoder.full_slen, 
                                           map_fluxes_sim_image.squeeze(0)[:, 0], 
                                           true_full_fluxes.squeeze(0)[:, 0], 
                                          n_elect_per_nmgy)
    
print('completeness: {:0.3f}'.format(completeness))
print('true positive rate: {:0.3f}'.format(tpr))

In [ ]:
true_locs, true_fluxes = image_statistics_lib.filter_params(true_full_locs.squeeze(),
                                                    true_full_fluxes.squeeze(0)[:, 0], 101, 5)

true_mags = image_statistics_lib.convert_nmgy_to_mag(true_fluxes / n_elect_per_nmgy)

In [ ]:
bool_indx = ((completeness_bool == 0) & \
    (true_mags >= 16.) & \
    (true_mags <= 16.5))    

In [ ]:
bool_indx.sum()

In [ ]:
true_locs[bool_indx]

In [ ]:
completeness_vec, mag_vec = \
    image_statistics_lib.get_completeness_vec(map_locs_sim_image.squeeze(), 
                                           true_full_locs.squeeze(), 
                                           star_encoder.full_slen, 
                                           map_fluxes_sim_image.squeeze(0)[:, 0], 
                                           true_full_fluxes.squeeze(0)[:, 0], 
                                             n_elect_per_nmgy)[0:2]

plt.plot(mag_vec[0:-1], completeness_vec, '--x', label = 'sim')
plt.xlabel('true log flux')
plt.ylabel('completeness')


In [ ]:
tpr_vec, mag_vec = \
    image_statistics_lib.get_tpr_vec(map_locs_sim_image.squeeze(), 
                                           true_full_locs.squeeze(), 
                                           star_encoder.full_slen, 
                                           map_fluxes_sim_image.squeeze(0)[:, 0], 
                                           true_full_fluxes.squeeze(0)[:, 0], 
                                    n_elect_per_nmgy)[0:2]


In [ ]:
plt.plot(mag_vec[0:-1], tpr_vec, '--x', label = 'sim')
plt.xlabel('est magnitude')
plt.ylabel('tpr')
